In [ ]:
# ProjectGurukul Face mask Detector

  #Import necessary packages
import cv2
import uuid
import mediapipe as mp

In [32]:
# Define mediapipe Face detector
face_detection = mp.solutions.face_detection.FaceDetection()

In [33]:
# Detection function for Face Mask Detection
def get_detection(frame):
    
    height, width, channel = frame.shape
    # Convert frame BGR to RGB colorspace
    imgRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Detect results from the frame
    result = face_detection.process(imgRGB)
    # Extract data from result
    try:
        for count, detection in enumerate(result.detections):
        # print(detection)
        # Extract bounding box information
            box = detection.location_data.relative_bounding_box
            x, y, w, h = int(box.xmin*width), int(box.ymin * height),int(box.width*width), int(box.height*height)
          
    # If detection is not available then pass
    except:
        pass
    return x, y, w, h

In [35]:
cap = cv2.VideoCapture(0)

In [36]:
count = 0
class_path = 'mask'
while True:
    _, frame = cap.read()
    img = frame.copy()
    try:
        # Make detection
        x, y, w, h = get_detection(frame)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 2)

        # Crop only the face part from the frame
        crop_img = img[y:y+h, x:x+w]
        filename = "Data/"+class_path+"/"+str(uuid.uuid4())+".jpg"

        # Save the image to the destination path
        cv2.imwrite(filename, crop_img)
        cv2.imshow("frame", crop_img)
        count+=1
    except:
        pass
    if cv2.waitKey(1) == ord('q') or count>=500:
        break

cap.release()
cv2.destroyAllWindows()

In [37]:
# import necessary packages
import os
import cv2
import time
from tqdm import tqdm
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Activation, Dropout

In [39]:
img_size = 100
datadir = r'data'  # root data directory
CATEGORIES = os.listdir(datadir)
print(CATEGORIES)

['mask', 'no_mask']


In [41]:
['no_mask', 'mask']
    # Define two empty list to contain image data
x, y = [], []
   
def PreProcess():
    for category in CATEGORIES:
        path = os.path.join(datadir, category)
        classIndex = CATEGORIES.index(category)
        print(path)
        for imgs in tqdm(os.listdir(path)):
            img_arr = cv2.imread(os.path.join(path, imgs))

              # resize the image
            resized_array = cv2.resize(img_arr, (img_size,img_size))
            cv2.imshow("images", resized_array)
            cv2.waitKey(1)
            resized_array = resized_array/255.0
            x.append(resized_array)
            y.append(classIndex)
          
PreProcess()
cv2.destroyAllWindows()

data/mask


100%|███████████████████████████████████████| 1000/1000 [00:16<00:00, 61.08it/s]


data/no_mask


100%|██████████████████████████████████████| 1000/1000 [00:06<00:00, 165.97it/s]


In [42]:
# Split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20,random_state=42)
  # Convert and resize the data to a numpy array
X_train = np.array(X_train).reshape(-1, img_size, img_size, 3)
y_train = np.array(y_train)
X_test = np.array(X_test).reshape(-1, img_size, img_size, 3)
y_test = np.array(y_test)

In [43]:
# Create the model architecture 
model = Sequential()
model.add(Conv2D(64,(3, 3), input_shape=(img_size, img_size, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(len(CATEGORIES)))
model.add(Activation('softmax'))
# compile the model
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', 
metrics=['accuracy'])
model.summary()

2023-03-05 10:41:37.499751: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/cmos/.local/lib/python3.10/site-packages/cv2/../../lib64:
2023-03-05 10:41:37.537562: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-05 10:41:37.548789: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cmos-HP-ProBook-4540s): /proc/driver/nvidia/version does not exist


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 64)        1792      
                                                                 
 activation (Activation)     (None, 98, 98, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 49, 49, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 256)       147712    
                                                                 
 activation_1 (Activation)   (None, 47, 47, 256)       0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 23, 23, 256)      0         
 2D)                                                    

In [44]:
batch_size = 32
epochs = 15
t1 = time.time()
# fit the model
model.fit(X_train, y_train, batch_size = batch_size, epochs=epochs,
validation_split=0.3, verbose = 1)
model.save('{}.h5'.format("model"))
t2 = time.time()
print('Time taken: ',t2-t1)

2023-03-05 10:42:03.291532: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 134400000 exceeds 10% of free system memory.


Epoch 1/15


2023-03-05 10:42:10.722506: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 78675968 exceeds 10% of free system memory.
2023-03-05 10:42:11.447328: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 19668992 exceeds 10% of free system memory.
2023-03-05 10:42:11.520103: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 72384512 exceeds 10% of free system memory.
2023-03-05 10:42:12.179312: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 17334272 exceeds 10% of free system memory.


35/35 [==============================] - 110s 3s/step - loss: 0.4471 - accuracy: 0.7955 - val_loss: 0.4047 - val_accuracy: 0.8417
Epoch 2/15
35/35 [==============================] - 100s 3s/step - loss: 0.2256 - accuracy: 0.9170 - val_loss: 0.2746 - val_accuracy: 0.8854
Epoch 3/15
35/35 [==============================] - 94s 3s/step - loss: 0.2091 - accuracy: 0.9304 - val_loss: 0.2438 - val_accuracy: 0.9021
Epoch 4/15
35/35 [==============================] - 99s 3s/step - loss: 0.2173 - accuracy: 0.9196 - val_loss: 0.2729 - val_accuracy: 0.8917
Epoch 5/15
35/35 [==============================] - 99s 3s/step - loss: 0.1876 - accuracy: 0.9330 - val_loss: 0.1801 - val_accuracy: 0.9375
Epoch 6/15
35/35 [==============================] - 99s 3s/step - loss: 0.1688 - accuracy: 0.9393 - val_loss: 0.1427 - val_accuracy: 0.9417
Epoch 7/15
35/35 [==============================] - 93s 3s/step - loss: 0.1246 - accuracy: 0.9554 - val_loss: 0.1924 - val_accuracy: 0.9187
Epoch 8/15
35/35 [===========

In [45]:
validation_loss, validation_accuracy = model.evaluate(X_test, y_test)

13/13 [==============================] - 11s 742ms/step - loss: 0.1753 - accuracy: 0.9500


In [ ]:
from tensorflow.keras.models import load_model

# load the model
model = load_model('model.h5')

In [ ]:
CATEGORIES = ['no_mask', 'mask']
cap = cv2.VideoCapture(0)
while True:
    _, frame = cap.read()
    img = frame.copy()
    try:
        x, y, w, h = get_detection(frame)
        crop_img = img[y:y+h, x:x+w]
        crop_img = cv2.resize(crop_img, (100, 100))
        crop_img = np.expand_dims(crop_img, axis=0)
        prediction = model.predict(crop_img)
        index = np.argmax(prediction)
        res = CATEGORIES[index]
        if index == 0:
            color = (0, 0, 255)
        else:
            color = (0, 255, 0)
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, res, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX,0.8, color, 2, cv2.LINE_AA)
    except:
        pass

    cv2.imshow("frame", frame)
    if cv2.waitKey(1) == ord('q'):
        break